# 모듈, 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install folium matplotlib mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 92.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
df_grid = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/unique_grid.csv')
df_grid.head(2)

,gid,lbl,val,geometry,gu
0,다사58a48a,6085.0,6085.0,"POLYGON ((958000 1948000, 958000 1948500, 9585...",강남구
1,다사58b48a,3446.0,3446.0,"POLYGON ((958500 1948000, 958500 1948500, 9590...",강남구


In [ ]:
pedestrian = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/도로시설물/서울시 보행자작동신호기 관련 정보.csv', encoding = 'cp949')
pedestrian.head(2)

,보행자작동신호기 관리번호,지주관리번호,방향 (공통),설치일,교체일,X좌표,Y좌표,제조업체,작업구분 (공통),표출구분 (공통),종류,신규정규화ID,상태 (공통),공사관리번호,보행자작동신호기 관리번호.1,이력ID,위치정보,공사형태 (공통)
0,29-0000000399,02-0000074701,90.0,NaN,NaN,200085.99351,554855.27478,NaN,1,2,0.0,4306821.0,001,2000-0000-000,29-000399,475,1,NaN
1,29-0000000584,02-0000081242,90.0,NaN,NaN,208627.53247,556488.58557,NaN,1,2,0.0,5379942.0,001,2000-0000-000,29-000584,516,1,NaN


# 데이터 정제
- 컬럼중에서 	보행자작동신호기 관리번호, X좌표, Y좌표 컬럼만 추출
- 널값 제외하고 값이 존재하는 데이터만 추출

In [ ]:
pedestrian.info()

In [ ]:
pedestrian = pedestrian.loc[ :, ['보행자작동신호기 관리번호', 'X좌표' , 'Y좌표']]
clean_pedestrian = pedestrian[pedestrian.X좌표.isnull() == False ]

In [ ]:
clean_pedestrian.info()
clean_pedestrian.head()

# pandas ⟶ geopandas

## 격자지도 데이터 프레임
```
기존에 .shp을 로드해서 csv로 저장했기 때문에 'geometry'열이 존재한다.

그러나 csv로 저장하면서 기존에 'geometry'열의 데이터 타입이 문자열로 변경되었다.

따라서 wkt.loads를 이용해서 문자열 타입을 폴리곤 타입으로 바꾸고, geodataframe을 구성할 때 좌표계 정보(epsg:5179)를 주었다.

```

In [ ]:
from shapely import wkt

df_grid['geometry'] = df_grid['geometry'].apply(wkt.loads)
geo_grids = gpd.GeoDataFrame(df_grid, crs='epsg:5179')

# 보행자작동신호기 데이터 프레임
```
기존에 .csv 파일이었기 때문에 'geometry'열이 존재하지 않는다.

따라서 geodataframe을 구성할 때 x좌표와 y좌표를 이용하여 'geometry' 컬럼을 만들어준다. 

그 후 to_crs() 함수를 이용해 좌표계를 5179 좌표계로 변경한다.

좌표계 설정 이유 : 좌표계의 종류가 많기 때문에 어떤 종류의 좌표계인지 명시해주어야 함

```

## geopandas 의 sjoin을 사용

- 정확히는 모르지만 공간에 따라 데이터를 join 하는 것 같음
- pandas.join과 유사 ( 다른 점: pandas의 join은 데이터가 같은 것끼리 엮지만 geopandas의 sjoin은 공간이 겹치면 엮는다 )

In [ ]:
geo_pedestrian = gpd.GeoDataFrame(
    clean_pedestrian, geometry=gpd.points_from_xy(clean_pedestrian['X좌표'], clean_pedestrian['Y좌표'])
)
geo_pedestrian.set_crs(epsg = 5186, inplace = True)
geo_pedestrian = geo_pedestrian.to_crs(epsg=5179)

In [ ]:
geo_pedestrian.explore(tiles='http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}', attr='Google')

In [ ]:
join_geo_df = gpd.sjoin(geo_pedestrian, geo_grids, how='left')
join_geo_df.shape

(2195, 9)

In [ ]:
# 피벗테이블 이용해서 격자지도별로 보행자작동신호기 카운트
pedestrian_by_grid = join_geo_df.pivot_table(index='gid', values='보행자작동신호기 관리번호', aggfunc='count')
pedestrian_by_grid.head(5)

,보행자작동신호기 관리번호
gid,
다사38a53a,1
다사39a51b,2
다사39a52b,4
다사39a53a,2
다사39b51b,6


In [ ]:
pedestrian_by_grid.sort_values(by='보행자작동신호기 관리번호', ascending=False).head(5)

,보행자작동신호기 관리번호
gid,
다사57a53a,28
다사44b47b,20
다사57a62a,19
다사53a50b,18
다사62b52b,17


In [ ]:
pedestrian_by_grid.to_csv('pedestrian_by_grid')